In [26]:
import pandas as pd
from neo4j import GraphDatabase

pd.set_option('display.max_columns',None)

df = pd.read_csv('./data/restaurant_reviews_withsentiment_null.csv')
df_loc = pd.read_csv('./data/restaurants_latest.csv')

In [27]:
df.head(100)

,Title,Body,Reviewer,Reviewer_ID,Reviewer_Level,Reviewer_NumReviews,ReviewDateTime,RestaurantName,Restaurant,Review,Food,RestaurantReview,FoodReview,RestaurantFeature(spacy),FoodFeature(spacy),RestaurantFeature(online),FoodFeature(online),RestaurantSentimentProba,FoodSentimentProba,OverallSentimentClass,Cuisine
0,Glad It’s Back!,We ordered two appetisers and four main dishes...,Sofina Ng,U0001,3.0,9.0,10m ago,Babasan by Uncle Kiisu,/babasan-by-uncle-kiisu,Glad It’s Back!. We ordered two appetisers and...,"beer,fruit",glad it’s back!. we ordered two appetisers and...,kronenbourg blanc beer was very enjoyable too....,"ordered,main,unique,boasting,manifested,creati...","enjoyable,sweet,mild",unique,"enjoyable,fruity,mild,strong,sweet",0.981540,0.805150,1,NaN
1,Yuzu Ochazuke ($24),This rice in soup dish is the first one I’ve h...,Jan L,U0002,3.0,10.0,5d ago,Babasan by Uncle Kiisu,/babasan-by-uncle-kiisu,Yuzu Ochazuke ($24). This rice in soup dish is...,rice,yuzu ochazuke ($24). boy did it impress! perfe...,this rice in soup dish is the first one i’ve h...,"impress,perfect,harmonious,fresh,generous,serv...","great,good","good,great,style","fresh,overpowering",0.963360,0.993274,1,NaN
2,Belly Roll Mee ($15),"The successor to Uncle Kiisu, Babasan’s menu d...",Dex Neo,U0003,8.0,958.0,Oct 24 at 5:42pm,Babasan by Uncle Kiisu,/babasan-by-uncle-kiisu,Belly Roll Mee ($15). The successor to Uncle K...,"crispy pork,mee siam,pork,pork belly",belly roll mee ($15). the successor to uncle k...,i was seriously wondering how they would pull ...,"dreamt,good,sounds,heavy,provided,cut,amazingl...","seriously wondering,pull,crispy,stuffed,tender...","affordable,good","crispy,heavy,icy,rich,spicy,stuffed,tangy,tast...",0.986915,0.959834,1,NaN
3,Samurai Kueh Pie Tee,This was like a little bomb that exploded with...,Komal Salve,U0004,7.0,208.0,Oct 23 at 10:06am,Babasan by Uncle Kiisu,/babasan-by-uncle-kiisu,Samurai Kueh Pie Tee. This was like a little b...,"fish,fish roe,kueh pie tee,pie,rice,roe,seafoo...",tee. this was like a little bomb that exploded...,samurai kueh pie i'm not a big fan of the seaf...,"little,exploded,eating,try,super worth","big,loved,flying,smooth","big,little","smooth,texture",0.959460,0.964552,1,NaN
4,Build Your Own Bowl (Small),Most salad/grain bowls serve up similar stuff ...,Nom Nom,U0005,3.0,15.0,Oct 25 at 10:13pm,YoBo,/yobo,Build Your Own Bowl (Small). Most salad/grain ...,"brown rice,cream,pumpkin,rice,salad",build your own bowl (small). but i really enjo...,most salad/grain bowls serve up similar stuff ...,"build,small,enjoyed,served","salad,serve,similar,creamy,paired,brown,roaste...","good,small","creamy,roasted",0.915978,0.972140,1,NaN
5,Healthy Warm Salad Bowls At Affordable Pricing,"I ordered 2 servings of honey soy, 1 serving o...",Ray L,U0006,2.0,1.0,Oct 18 at 3:40pm,YoBo,/yobo,Healthy Warm Salad Bowls At Affordable Pricing...,"aglio olio,brussel sprout,fusilli,mushroom,nut...",this was all done using their online portal an...,healthy warm salad bowls at affordable pricing...,"online,paid,nice,late,eat,definitely recommend","healthy,warm,salad,affordable,ordered,serving,...","affordable,nice,warm","garlicky,healthy,light,nutty,strong,warm",0.925739,0.959115,1,NaN
6,Excellent Value For Money In CBD,Away with standardised salad/grain bowls in CB...,shirley L,U0007,2.0,1.0,Oct 18 at 12:47pm,YoBo,/yobo,Excellent Value For Money In CBD. Away with st...,"nut,salad",excellent value for money in cbd. you can have...,away with standardised salad/grain bowls in cb...,"excellent,kcal,recommended,including,recommend","away standardised,salad,offers,pay,add,excelle...",excellent,NaN,0.941941,0.878147,1,NaN
7,For Delicious Build-Your-Own-Bowls in the CBD,Make note of this spot on Cecil Street for qui...,Burpple Guides,U0008,NaN,NaN,Oct 16 at 4:16pm,YoBo,/yobo,For Delicious Build-Your-Own-Bowls in the CBD....,"brown rice,chicken,cream,kale chip,pho,rice",for delicious build-your-own-bowls in the cbd....,keep it healthy with japane

In [29]:
df_loc.head()

,Unnamed: 0,Title,Link,Location,Latitude,Longitude,NumReviews,PriceRange,Tags,Categories,Address
0,0,Babasan by Uncle Kiisu,/babasan-by-uncle-kiisu,Clarke Quay,1.286034,103.849552,4,~$30/pax,['BEYOND'],"['Local Delights', ' Japanese', ' 1-for-1 Deal...",50 Circular Road Singapore 049405
1,1,Enjoy Eating House and Bar,/enjoy-eating-house-and-bar,Farrer Park,1.313191,103.859697,131,~$15/pax,"['BEYOND', 'Featured In 3 Guides']","['Bars', ' Local Delights', ' Zi Char', ' Thai...",383 Jalan Besar Kam Leng Hotel Singapore 209001
2,2,Wu Pao Chun,/wu-pao-chun-bakery,City Hall,1.292939,103.851480,79,~$13/pax,"['BEYOND', 'Featured In 1 Guide']","['Bread', ' 1-for-1 Deals', ' Burpple Beyond']",13 Stamford Road #01-19/20 Capitol Singapore ...
3,3,The Salted Plum (Circular Road),/thesaltedplumsg,Raffles Place,1.287783,103.848887,215,~$20/pax,"['BEYOND', 'Hot 100 2019']","['Taiwanese', ' Cheap & Good', ' 1-for-1 Deals...",10 Circular Road Singapore 049366
4,4,Rakki Bowl,/rakki-bowl,Tanjong Pagar,1.278173,103.843013,293,~$25/pax,"['BEYOND', 'Hot 100 2019', 'Featured In 2 Guid...","['Chirashi', ' Japanese', ' 1-for-1 Deals']",72 Duxton Road Singapore 089531


In [30]:
df_loc.rename(columns={'Link':'Restaurant'},inplace=True)
df = df.merge(df_loc[['Restaurant','Address']], how='left', on='Restaurant')

In [31]:
df['Review'] = df['Title'] + ":" + df['Body']

In [32]:
df.fillna("",inplace=True)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63921 entries, 0 to 63920
Data columns (total 22 columns):
Title                        63921 non-null object
Body                         63921 non-null object
Reviewer                     63921 non-null object
Reviewer_ID                  63921 non-null object
Reviewer_Level               63921 non-null object
Reviewer_NumReviews          63921 non-null object
ReviewDateTime               63921 non-null object
RestaurantName               63921 non-null object
Restaurant                   63921 non-null object
Review                       63921 non-null object
Food                         63921 non-null object
RestaurantReview             63921 non-null object
FoodReview                   63921 non-null object
RestaurantFeature(spacy)     63921 non-null object
FoodFeature(spacy)           63921 non-null object
RestaurantFeature(online)    63921 non-null object
FoodFeature(online)          63921 non-null object
RestaurantSentimentProba

In [34]:
df.head()

,Title,Body,Reviewer,Reviewer_ID,Reviewer_Level,Reviewer_NumReviews,ReviewDateTime,RestaurantName,Restaurant,Review,Food,RestaurantReview,FoodReview,RestaurantFeature(spacy),FoodFeature(spacy),RestaurantFeature(online),FoodFeature(online),RestaurantSentimentProba,FoodSentimentProba,OverallSentimentClass,Cuisine,Address
0,Glad It’s Back!,We ordered two appetisers and four main dishes...,Sofina Ng,U0001,3,9,10m ago,Babasan by Uncle Kiisu,/babasan-by-uncle-kiisu,Glad It’s Back!:We ordered two appetisers and ...,"beer,fruit",glad it’s back!. we ordered two appetisers and...,kronenbourg blanc beer was very enjoyable too....,"ordered,main,unique,boasting,manifested,creati...","enjoyable,sweet,mild",unique,"enjoyable,fruity,mild,strong,sweet",0.981540,0.805150,1,,50 Circular Road Singapore 049405
1,Yuzu Ochazuke ($24),This rice in soup dish is the first one I’ve h...,Jan L,U0002,3,10,5d ago,Babasan by Uncle Kiisu,/babasan-by-uncle-kiisu,Yuzu Ochazuke ($24):This rice in soup dish is ...,rice,yuzu ochazuke ($24). boy did it impress! perfe...,this rice in soup dish is the first one i’ve h...,"impress,perfect,harmonious,fresh,generous,serv...","great,good","good,great,style","fresh,overpowering",0.963360,0.993274,1,,50 Circular Road Singapore 049405
2,Belly Roll Mee ($15),"The successor to Uncle Kiisu, Babasan’s menu d...",Dex Neo,U0003,8,958,Oct 24 at 5:42pm,Babasan by Uncle Kiisu,/babasan-by-uncle-kiisu,Belly Roll Mee ($15):The successor to Uncle Ki...,"crispy pork,mee siam,pork,pork belly",belly roll mee ($15). the successor to uncle k...,i was seriously wondering how they would pull ...,"dreamt,good,sounds,heavy,provided,cut,amazingl...","seriously wondering,pull,crispy,stuffed,tender...","affordable,good","crispy,heavy,icy,rich,spicy,stuffed,tangy,tast...",0.986915,0.959834,1,,50 Circular Road Singapore 049405
3,Samurai Kueh Pie Tee,This was like a little bomb that exploded with...,Komal Salve,U0004,7,208,Oct 23 at 10:06am,Babasan by Uncle Kiisu,/babasan-by-uncle-kiisu,Samurai Kueh Pie Tee:This was like a little bo...,"fish,fish roe,kueh pie tee,pie,rice,roe,seafoo...",tee. this was like a little bomb that exploded...,samurai kueh pie i'm not a big fan of the seaf...,"little,exploded,eating,try,super worth","big,loved,flying,smooth","big,little","smooth,texture",0.959460,0.964552,1,,50 Circular Road Singapore 049405
4,Build Your Own Bowl (Small),Most salad/grain bowls serve up similar stuff ...,Nom Nom,U0005,3,15,Oct 25 at 10:13pm,YoBo,/yobo,Build Your Own Bowl (Small):Most salad/grain b...,"brown rice,cream,pumpkin,rice,salad",build your own bowl (small). but i really enjo...,most salad/grain bowls serve up similar stuff ...,"build,small,enjoyed,served","salad,serve,similar,creamy,paired,brown,roaste...","good,small","creamy,roasted",0.915978,0.972140,1,,182 Cecil Street #02-14 Frasers Tower Singapor...


In [43]:
from py2neo import Graph, Node, Relationship
# Input the user name and password set in the neo4j graph
graph = Graph("bolt://localhost:7687", auth=("neo4j","123456"))
graph.delete_all()

In [44]:
for index, row in df.iterrows():
    
    # Create notes for review, reviewer, restaurant, food review and restaurant review
    review = Node('Review', 
                  review = row['Review'],
                  overall_sentiment = row['OverallSentimentClass']) 

    reviewer = Node('Reviewer', 
                    name = row['Reviewer_ID'],
                    reviewer_level = row['Reviewer_Level'], 
                    no_of_reviews= row['Reviewer_NumReviews'])
    
    restaurant = Node('Restaurant', 
                  name = row['RestaurantName'],
                  address= row['Address'],
                  cuisine = row['Cuisine'])
    
#     foodreview = Node('Food Review', 
#                       food_review = row['FoodReview'], 
#                       food_sentiment = row['FoodSentimentProba'],
#                       features = row['FoodFeature(online)'])
    
#     restaurantreview = Node('Restaurant Review', 
#                             restaurant_review = row['RestaurantReview'], 
#                             restaurant_sentiment = row['RestaurantSentimentProba'], 
#                             features = row['RestaurantFeature(online)'])
    
    temp_foodfeatures = row['FoodFeature(online)'].split(',')
    for feature in temp_foodfeatures:
        while "," in feature:
            feature = feature.split(",")
            for feature in set(feature):
                foodfeature = Node('Food Features', feature = feature)
        else:
            foodfeature = Node('Food Features', feature = feature)
            
    temp_restaurantfeatures = row['RestaurantFeature(online)'].split(',')
    for feature in temp_restaurantfeatures:
        while "," in feature:
            feature = feature.split(",")
            for feature in set(feature):
                restaurantfeature = Node('Restaurant Features', feature = feature)
        else:
            restaurantfeature = Node('Restaurant Features', feature = feature)
    
    # Remove duplicates for reviewer and restaurant
    graph.create(review)
    graph.merge(reviewer, 'Reviewer','name')
    graph.merge(restaurant, 'Restaurant', 'name')
    graph.merge(foodfeature, 'Food Features', 'feature')
    graph.merge(restaurantfeature, 'Restaurant Features', 'feature')

    # for each food in review, create a node
    temp_food = row['Food'].split(',')
    for food in temp_food:
        if "," in food:
            food = food.split(",")
            for food in set(food): 
                food_node = Node('Food', 
                            name = food, 
                            features = row['FoodFeature(online)'])
                
                # Remove duplicates for food
                graph.merge(food_node,'Food','name')
                
                # Create relationships for review [contains]-> food, food [servedby]-> restaurant
                review_contains_food = Relationship(review, "Contains", food_node,
                   food_sentiment = row['FoodSentimentProba'],
                   features=row['FoodFeature(online)'])
                
                food_servedby_restaurant = Relationship(food_node, "Servedby", restaurant)
                food_has_feature = Relationship(food_node, "Has", foodfeature)
                
                graph.create(review_contains_food)
                graph.create(food_servedby_restaurant)
                graph.create(food_has_feature)
                
        else:
            food_node = Node('Food', 
                            name = food, 
                            features = row['FoodFeature(online)'])
                      
            graph.merge(food_node,'Food','name')

            review_contains_food = Relationship(review, "Contains", food_node,
                            food_sentiment = row['FoodSentimentProba'],
                            features=row['FoodFeature(online)'])

            food_servedby_restaurant = Relationship(food_node, "Servedby", restaurant)
            food_has_feature = Relationship(food_node, "Has", foodfeature)
            
            graph.create(review_contains_food)
            graph.create(food_servedby_restaurant)
            graph.create(food_has_feature)
            
                
    # Create relationships for reviewer [wrote]-> review, review [about]-> restaurant
    reviewer_wrote_review = Relationship(reviewer, "Wrote", review)
    
    review_about_restaurant = Relationship(review, "About", restaurant, 
                                           restaurant_sentiment = row['RestaurantSentimentProba'], 
                                           features=row['RestaurantFeature(online)'])
    
    restaurant_has_feature = Relationship(restaurant, "Has", restaurantfeature)
    
    graph.create(reviewer_wrote_review)
    graph.create(review_about_restaurant)  
    graph.create(restaurant_has_feature)

In [46]:
output = graph.run("MATCH (n) RETURN n").to_data_frame()

Failed to write data to connection ('localhost', 7687) (Address(host='127.0.0.1', port=7687)); ("10054; '远程主机强迫关闭了一个现有的连接。'; None; 10054; None")


In [47]:
output.to_csv('./data/graph_output.csv')

In [48]:
output.shape

(72550, 1)